<a href="https://colab.research.google.com/github/kiing-dom/CS4287/blob/main/Assignment2CNN/CS4287_Assign2_20243561_20250762.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS4287 CNN**

In [35]:
# 20243561 - Dominion Gbadamosi
# Runs

In [36]:
#Importing Necessary Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import os


## Data Visualization



In [37]:
# using pandas to read the library
dataset = "all_letters_info.csv"
df = pd.read_csv(dataset)

#display the header of the dataset
display(df.head())

,letter,label,file,background
0,а,1,01_01.png,0
1,а,1,01_02.png,0
2,а,1,01_03.png,0
3,а,1,01_04.png,0
4,а,1,01_05.png,0


In [38]:
#folder path where the images are stored
images_folder = "all_letters_image/all_letters_image/"

In [39]:
labels = df['label'] #corresponding to the "label" column in the csv file
file_names = df['file'] #corresponds to the "file" column in the csv file

## Data Preprocessing

In [40]:
# Preprocessing data
le = LabelEncoder()
labels = le.fit_transform(labels)

#Image Preprocessing
images = []

for file_name in file_names:
  # create the full file path
  file_path = os.path.join(images_folder, file_name)

  # load and preprocess the image
  img = tf.keras.preprocessing.image.load_img(file_path, target_size=(32,32))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = img / 255.0 #Normalization of pixel values to the range [0, 1] to make values more suitable for neural network
  images.append(img)

images = np.array(images)

#splitting the dataset into training & test data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)